# Correr experimentos
En este archivo está el código para correr los experimentos y escribir los resultados en archivos CSV.
> Los archivos se guardan en la carpeta _resultados_.

In [32]:
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

A continuación leemos los datasets en dataframes de Pandas.

In [33]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read();

In [63]:
berlin52 = pd.read_csv("instancias/mejor-caso-bt-f/indice.csv");

In [34]:
df_aleatorio = pd.read_csv("instancias_aleatorias/indice.csv")

La siguiente función sirve para correr el código sobre una instancia ejecutando un método en particular.

- **VMC** : vecino mas cercano
- **AGM** : arbol generador minimo
- **I** : incersion

In [76]:
def correr_experimento(metodo, archivo_instancia):
    # Leer archivo de la instancia.
    instancia = leer_instancia(archivo_instancia)
    
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["../tp2", metodo], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar.
    process.stdin.write(instancia)
    process.stdin.flush()

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise(F"Hubo un error en la experimentacion para el algoritmo: {algoritmo} con la instancia {archivo_instancia}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempo_de_ejecucion = float(process.stderr.read());
    costo = process.stdout.read() 
    #print(process.stdout.read()) # resultado del metodo
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    costo = costo.split(" ")[1]
    costo = costo.split("\n")[0]
    res = (tiempo,costo)
    
    return res;




## Corremos los experimentos
Vamos a guardar una tabla con las ejecuciones y sus respectivos tiempos.

In [77]:
experimentos = [];

## Experimento aleatorio


In [78]:
for i in range(0, df_aleatorio.shape[0]):
    fila = df_aleatorio.iloc[i];
    experimentos.append([fila["dataset"], fila["n"],"AGM", fila["archivo"]]);
    experimentos.append([fila["dataset"], fila["n"],"VMC", fila["archivo"]]);

## Ejemplo TP1

## Experimento Fuerza Bruta : Pocas/Muchas soluciones

In [47]:
for i in range(0, df_fbms.shape[0]):
    fila = df_fbms.iloc[i];
    n = fila["n"] 
    if (n <= 25):
        experimentos.append([fila["dataset"], fila["n"], fila["R"], "FB", fila["archivo"]]);
    
for i in range(0, df_fbps.shape[0]):
    fila = df_fbps.iloc[i];
    n = fila["n"] 
    if (n <= 25):
        experimentos.append([fila["dataset"], fila["n"], fila["R"], "FB", fila["archivo"]]);

## Ejecutar los experimentos y guardar los resultados en un archivo CSV.
Este paso puede tardar unos minutos hasta terminar de ejecutarse.

In [79]:
columnas = ["dataset", "n", "metodo", "tiempo","costo"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    costo = 0
    tiempos = []
    resultados = []
    for i in range(0, T):
        t = correr_experimento(experimento[2], experimento[3])[0]
        costo = correr_experimento(experimento[2], experimento[3])[1]
        tiempos.append(t);
    tiempo = np.median(tiempos);
    filas.append([experimento[0], experimento[1], experimento[2], tiempo,costo]);


'Experimento: 158/158'

In [24]:
# ANTERIOR

columnas = ["dataset", "n", "R", "metodo", "tiempo"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos = []
    resultados = []
    for i in range(0, T):
        t = correr_experimento(experimento[3], experimento[4])
        tiempos.append(t);
    tiempo = np.median(tiempos);
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], tiempo]);


In [80]:
df_resultados = pd.DataFrame(filas, columns=columnas);

df_resultados.to_csv("resultados/resultado.csv", index=False, header=True);

In [81]:
df_resultados

,dataset,n,metodo,tiempo,costo
0,dataset_aleatorias,10,AGM,2.74717,1253
1,dataset_aleatorias,10,VMC,2.74717,1101
2,dataset_aleatorias,20,AGM,2.74717,1106
3,dataset_aleatorias,20,VMC,2.74717,960
4,dataset_aleatorias,30,AGM,2.74717,2102
...,...,...,...,...,...
153,dataset_aleatorias,770,VMC,2.74717,9059
154,dataset_aleatorias,780,AGM,2.74717,59661
155,dataset_aleatorias,780,VMC,2.74717,9550
156,dataset_aleatorias,790,AGM,2.74717,62209
